### Dictionary and set as hash table
1. dict is a hash table. The hash value is calculated using the key.  
2. When a['x'] = b is done, the special method a.\__hash__('x') is done and the key/value pair is stored in the resulting hash bucket.  
3. Python hash functions (the following functions return the hash key. This key is further sent to a hash function to get the hash bucket value):    
    - Integers: returns the integer, or -2 if integer == -1  
    - Strings:
    ```
    class string:
        def __hash__(self):
            if not self:
                return 0 # empty
            value = ord(self[0]) << 7
            for char in self:
                value = c_mul(1000003, value) ^ ord(char)
            value = value ^ len(self)
            if value == -1:
                value = -2
            return value
    ```
    (c_mul essentially implements overflow in Python)

    - Tuples:
    ```
    class tuple:
        def __hash__(self):
            value = 0x345678
            for item in self:
                value = c_mul(1000003, value) ^ hash(item)
            value = value ^ len(self)
            if value == -1:
                value = -2
            return value
    ```

    - Decimals:
    ```
    def __hash__(self):
            """x.__hash__() <==> hash(x)"""
            # Decimal integers must hash the same as the ints
            #
            # The hash of a nonspecial noninteger Decimal must depend only
            # on the value of that Decimal, and not on its representation.
            # For example: hash(Decimal('100E-1')) == hash(Decimal('10')).
            if self._is_special:
                if self._isnan():
                    raise TypeError('Cannot hash a NaN value.')
                return hash(str(self))
            if not self:
                return 0
            if self._isinteger():
                op = _WorkRep(self.to_integral_value())
                # to make computation feasible for Decimals with large
                # exponent, we use the fact that hash(n) == hash(m) for
                # any two nonzero integers n and m such that (i) n and m
                # have the same sign, and (ii) n is congruent to m modulo
                # 2**64-1.  So we can replace hash((-1)**s*c*10**e) with
                # hash((-1)**s*c*pow(10, e, 2**64-1).
                return hash((-1)**op.sign*Page on Op*pow(10, op.exp, 2**64-1))
            # The value of a nonzero nonspecial Decimal instance is
            # faithfully represented by the triple consisting of its sign,
            # its adjusted exponent, and its coefficient with trailing
            # zeros removed.
            return hash((self._sign,
                         self._exp+len(self._int),
                         self._int.rstrip('0')))
    ```     

### Magic/Special methods
1. When you create an instance x of a class A with the statement "x = A()", Python will do the necessary calls to \__new__ and \__init__.  
2. If we have an expression "x + y" and x is an instance of class K, then Python will check the class definition of K. If K has a method \__add__ it will be called with x.\__add__(y), otherwise we will get an error message.  
3. \__hash__ is a method in the classes dict and set.  

### \__ init__.py
The \__init__.py files are required to make Python treat the 
directories as containing packages; this is done to prevent 
directories with a common name, such as string, from 
unintentionally hiding valid modules that occur later on the
module search path. In the simplest case,__init__.py can just
be an empty file, but it can also execute initialization code
for the package or set the __all__ variable, described later.